## libs

In [1]:
import io
import random
import string 
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)

import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt')

2021-03-17 07:53:58 INFO: Downloading default packages for language: pt (Portuguese)...
2021-03-17 07:53:58 INFO: File exists: /home/silvio/stanza_resources/pt/default.zip.
2021-03-17 07:54:01 INFO: Finished downloading models and saved to /home/silvio/stanza_resources.
2021-03-17 07:54:02 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2021-03-17 07:54:02 INFO: Use device: gpu
2021-03-17 07:54:02 INFO: Loading: tokenize
2021-03-17 07:54:05 INFO: Loading: mwt
2021-03-17 07:54:05 INFO: Loading: pos
2021-03-17 07:54:05 INFO: Loading: lemma
2021-03-17 07:54:05 INFO: Loading: depparse
2021-03-17 07:54:06 INFO: Done loading processors!


## Corpus

Sentenças que o chatbot vai usar como base de conhecimento

In [2]:
f=open('/home/silvio/soci.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()#lowercase

# carregando stop words em portugues

In [3]:
sw_pt=nltk.corpus.stopwords.words('portuguese') 

In [4]:
sw_pt

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

Pré-processamento básico para atacar desafios NLP:

* Converter todo o texto em maiúsculas ou minúsculas

* Tokenização

O tokenizador NLTK permite
* remover ruidos: tudo que nao e letra ou número da linguagem
* remover stop words
* Stemming: Stemming é o processo de redução de palavras flexionadas (ou às vezes derivadas) à sua raiz, forma de base ou raiz - geralmente uma forma de palavra escrita.

* Lematização: variante do Stemming. A principal diferença entre eles é que, frequentemente, o radical pode criar palavras inexistentes, ao passo que os lemas são palavras reais.  Exemplos de lematização são que “correr” é uma forma base para palavras como “correr” ou “correu” 

## Tokenização

In [26]:
sent_tokens = nltk.sent_tokenize(raw)# transformando em lista de sentencas

In [6]:
sent_tokens[1]

'por sua forma\nteórica, porém, o socialismo começa apresentando-se como uma continuação, mais desenvolvida e mais\nconseqüente, dos princípios proclamados pelos grandes pensadores franceses do século xviii.'

In [7]:
sent_tokens[10]

'os grandes pensadores do século xviii, como\ntodos os seus predecessores, não podiam romper as fronteiras que sua própria época lhes impunha.'

# retirando raiz da palavras - stem

In [8]:
from nltk.stem.snowball import SnowballStemmer

Stemmer=SnowballStemmer("portuguese")
Stemmer.stem("obtido")

'obtid'

In [9]:
Stemmer.stem(sent_tokens[10])

'os grandes pensadores do século xviii, como\ntodos os seus predecessores, não podiam romper as fronteiras que sua própria época lhes impunha.'

# lemmatização

In [10]:
txt="Não, minha miúda no sentido que és como uma irmã para mim."
txt="todos todas todo partido partida partidao partideiro partidos"
txt=sent_tokens[10]
lemma = ""
for sent in nlp(txt).sentences:
    for word in sent.words:
        lemma += word.lemma + "\t"

print(txt)
print(lemma)

os grandes pensadores do século xviii, como
todos os seus predecessores, não podiam romper as fronteiras que sua própria época lhes impunha.
o	grande	pensador	de	o	século	xviii	,	como	todo	o	seu	predecessor	,	não	poder	romper	o	fronteira	que	seu	próprio	época	eles	impor	.	


# preparando sentença 

In [11]:
from nltk.stem.snowball import SnowballStemmer

Stemmer=SnowballStemmer("portuguese")
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemTokens(tokens):
    return [Stemmer.stem(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [12]:
print(" original: " , sent_tokens[1])
print("")
print(" transformado: " , LemNormalize(sent_tokens[1]))

 original:  por sua forma
teórica, porém, o socialismo começa apresentando-se como uma continuação, mais desenvolvida e mais
conseqüente, dos princípios proclamados pelos grandes pensadores franceses do século xviii.

 transformado:  ['por', 'sua', 'form', 'teóric', 'porém', 'o', 'social', 'comec', 'apresentandos', 'com', 'uma', 'continu', 'mais', 'desenvolv', 'e', 'mais', 'consequent', 'dos', 'princípi', 'proclam', 'pel', 'grand', 'pensador', 'frances', 'do', 'sécul', 'xvii']


## Gerando Respostas

### Bag of Words
* Reduzir a senteças a conteúdos que sejam significativos para a tokenização (sensível a frequÊncia da palavra)

### TF-IDF ( Term Frequency-Inverse Document Frequency) 
* Penaliza termos que aprecem com muita frequência

### similaridade por cosseno
* Mede similaridade entre dois textos, com base no seus vetores

To generate a response from our bot for input questions, the concept of document similarity will be used. We define a function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”

In [36]:
def response(user_response):
    robo_response=''
    #adiciona a pergunta do usuário a lista de sentenças
    sent_tokens.append(user_response)
    
    # normaliza a expressão de entrada
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=sw_pt)
    
    # aplica função tfidf a todas as sentenças
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #print(tfidf)
    #obtem similaridade entre a sentença que foi incluida tfidf[-1] e todas as outras sentenças
    vals = cosine_similarity(tfidf[-1], tfidf)
    
    print(vals.shape)
    print('vals: ',vals)
    print('vals sort: ',vals.argsort())
    idx=vals.argsort()[0][-2]
    print('idx: ',idx)
    
    # Verificar se melhor similaridade encontrada é 0
    # se for responde como não entendi
    flat = vals.flatten()
    flat.sort()
    print('flat ', flat)
    req_tfidf = flat[-2]
    print('req_tfidf ', req_tfidf)
    
    if(req_tfidf==0):
        robo_response=robo_response+"não entendi"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
    return robo_response

In [39]:
Q = "O que é agumon?"
Ql=Q.lower()
A=response(Ql)
print(A)
sent_tokens.remove(Ql)

(1, 539)
vals:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.

# Rotina para obter entrada e calcular saída mais similar

In [ ]:
flag=True
print("chatbot")

while(flag==True):
    # obtem frase dita pelo usuário
    user_response = input()
    user_response=user_response.lower()
    # se não foi sair
    if(user_response!='sair'):
        print("ROBO: ",end="")
        print(response(user_response))
        sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: saindo")